In [16]:
import pandas as pd
import numpy as np
import ast
author_data = pd.read_csv(r'C:\Users\khali\OneDrive - Danmarks Tekniske Universitet\6. semester\Computational Social Science\Computational-Social-Science-Exercises\author_df.csv')
paper_data = pd.read_csv(r'C:\Users\khali\OneDrive - Danmarks Tekniske Universitet\6. semester\Computational Social Science\Computational-Social-Science-Exercises\paper_df.csv')

author_id_list_x = [119457888,50013417]

def TopField(author_data,author_id_list):

    return list(author_data[author_data['authorId'].isin(author_id_list)].get('field'))

field_data = []
for (count, paper_id) in enumerate(paper_data.get('authorIds')):
    x = ast.literal_eval(paper_id)
    x = [int(i) for i in x]
    field_data_x = TopField(author_data, x)
    if count % 10000 == 0:
        print(count)
    field_data.append(field_data_x)
    

field_data_pd = pd.DataFrame({'author_field': field_data})

paper_data['author_field'] = field_data_pd

#This takes a while to run, so better to save as new csv file
#paper_data.to_csv(r'C:\Users\khali\OneDrive - Danmarks Tekniske Universitet\6. semester\Computational Social Science\Computational-Social-Science-Exercises\paper.....csv')

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000


In [158]:
import pandas as pd
import numpy as np
import ast
# load paper data with author field
paper_data = pd.read_csv(r'C:\Users\khali\OneDrive - Danmarks Tekniske Universitet\6. semester\Computational Social Science\Computational-Social-Science-Exercises\paper_w_author_field_df.csv')


In [176]:
css_df = pd.DataFrame(columns=['Unnamed: 0', 'paperId', 'title', 'year', 'externalId.DOI',
       'citationCount', 'authorIds', 'fields', 'author_field'])

In [177]:
#css_data is paper data 
social_science_fields = ['Political Science', 'Sociology', 'Economics']
quantitative_fields = ['Mathematics','Physics','Computer Science']


for count in range(len(paper_data['author_field'])):
    
    #print('count',count)
    row = paper_data.iloc[count]

    auth_field = ast.literal_eval(row.author_field.replace('nan,', '').replace('nan]', ']').replace('[nan', '[').replace('[nan]', '[]'))

    field = ast.literal_eval(row.author_field.replace('nan,', '').replace('nan]', ']').replace('[nan', '[').replace('[nan]', '[]'))
    
    id_DOI = "skip" if pd.isna(row['externalId.DOI']) else ast.literal_eval(row['externalId.DOI'])
    
    if (auth_field == []) or (field == []) or (id_DOI == "skip"):
        continue
    
    
    criterias =     ([x for x in field if x in social_science_fields]) and \
                    (([x for x in field if x in quantitative_fields]) or ([x for x in auth_field if x in quantitative_fields])) and \
                    (row.year >= 2009) and \
                    (len(ast.literal_eval(row['authorIds'])) <= 10) and \
                    ('Biology' not in field) and \
                    (id_DOI.get('DOI') != None)
    
    if criterias:
        #add row if criteria is met:
        css_df = pd.concat([css_df, pd.DataFrame([row])], axis=0, ignore_index = True)
        print(f'appended count {count} to dataframe')



appended count 263 to dataframe
appended count 1672 to dataframe
appended count 1673 to dataframe
appended count 1676 to dataframe
appended count 3607 to dataframe
appended count 3608 to dataframe
appended count 3614 to dataframe
appended count 3615 to dataframe
appended count 3616 to dataframe
appended count 3619 to dataframe
appended count 3620 to dataframe
appended count 3621 to dataframe
appended count 3636 to dataframe
appended count 4984 to dataframe
appended count 4985 to dataframe
appended count 5032 to dataframe
appended count 5548 to dataframe
appended count 5561 to dataframe
appended count 5562 to dataframe
appended count 5570 to dataframe
appended count 6491 to dataframe
appended count 6637 to dataframe
appended count 6639 to dataframe
appended count 6641 to dataframe
appended count 6643 to dataframe
appended count 6644 to dataframe
appended count 6648 to dataframe
appended count 6649 to dataframe
appended count 6650 to dataframe
appended count 6652 to dataframe
appended co

In [178]:
#save css_df as csv file
#css_df.to_csv(r'C:\Users\khali\OneDrive - Danmarks Tekniske Universitet\6. semester\Computational Social Science\Computational-Social-Science-Exercises\css_df.csv')

In [181]:
len(css_df.year)

3223

In [319]:
import pandas as pd
from collections import defaultdict

author_data = pd.read_csv(r'C:\Users\khali\OneDrive - Danmarks Tekniske Universitet\6. semester\Computational Social Science\Computational-Social-Science-Exercises\author_df.csv',dtype=str)
css_df = pd.read_csv(r'C:\Users\khali\OneDrive - Danmarks Tekniske Universitet\6. semester\Computational Social Science\Computational-Social-Science-Exercises\css_df.csv',dtype=str)
css_df.drop(columns=['Unnamed: 0.1'],inplace=True)
css_df['year'] = css_df['year'].astype(float).astype(int)
css_df['citationCount'] = css_df['citationCount'].astype(float).astype(int)



In [320]:
author_data['authorId'][1
]

'3276812'

In [321]:
css_df.sort_values(by=['citationCount'],ascending=False)['title'].iloc[0:10]

1284    Conducting behavioral research on Amazon’s Mec...
1253    A 61-million-person experiment in social influ...
2794    Exposure to ideologically diverse news and opi...
2803    Automated Hate Speech Detection and the Proble...
291     Estimation and Inference of Heterogeneous Trea...
28      Commentary on Ferguson, et al., “Impact of Non...
2497    How Censorship in China Allows Government Crit...
2687    Inherent Trade-Offs in the Fair Determination ...
49      Multiscale mobility networks and the spatial s...
1235    Filter Bubbles, Echo Chambers, and Online News...
Name: title, dtype: object

In [324]:
#make dictionary for collab network
import ast
collab_network = {}

for i in range(len(css_df)):
    paper_authorId = ast.literal_eval(css_df.iloc[i]['authorIds'])

    for auth1 in paper_authorId:
        for auth2 in paper_authorId:
            if auth1 != auth2:
                if (auth1,auth2) in collab_network:
                    collab_network[(auth1,auth2)] += 1
                else:
                    collab_network[(auth1,auth2)] = 1

edgelist = [(x[0][0], x[0][1], x[1]) for x in collab_network.items()]

In [327]:
print(len(edgelist))
edgelist

18248


[('71958542', '50657181', 1),
 ('71958542', '1573630453', 1),
 ('71958542', '1573630578', 1),
 ('71958542', '48988567', 1),
 ('71958542', '2053577568', 1),
 ('50657181', '71958542', 1),
 ('50657181', '1573630453', 1),
 ('50657181', '1573630578', 1),
 ('50657181', '48988567', 1),
 ('50657181', '2053577568', 1),
 ('1573630453', '71958542', 1),
 ('1573630453', '50657181', 1),
 ('1573630453', '1573630578', 1),
 ('1573630453', '48988567', 1),
 ('1573630453', '2053577568', 1),
 ('1573630578', '71958542', 1),
 ('1573630578', '50657181', 1),
 ('1573630578', '1573630453', 1),
 ('1573630578', '48988567', 1),
 ('1573630578', '2053577568', 1),
 ('48988567', '71958542', 1),
 ('48988567', '50657181', 1),
 ('48988567', '1573630453', 1),
 ('48988567', '1573630578', 1),
 ('48988567', '2053577568', 1),
 ('2053577568', '71958542', 1),
 ('2053577568', '50657181', 1),
 ('2053577568', '1573630453', 1),
 ('2053577568', '1573630578', 1),
 ('2053577568', '48988567', 1),
 ('145883179', '2703478', 2),
 ('1458831

In [328]:
ast.literal_eval(author_data[author_data['authorId'] == '50657181']['aliases'].values[0])
#author_data[author_data['authorId'] == 50657181]['name'].values[0]

['Zhang Shuo', 'Zhan Shuo']

In [341]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import ast
from collections import defaultdict

G = nx.Graph()
G.add_weighted_edges_from(edgelist)



In [342]:
#find median citation count of authors in css_df:
css_citation_count = {}
css_first_paper_year = {}
css_author_fullname = {}
css_citation_count_median = {}
css_num_papers = {}
css_author_field = {}

for i in range(len(css_df)):
    paper_authorId = ast.literal_eval(css_df.iloc[i]['authorIds'])
    paper_first_paper_year = css_df.iloc[i]['year']
    for auth in paper_authorId:
        auth = str(auth)
        if auth in css_first_paper_year:
            css_first_paper_year[auth] = min(css_first_paper_year[auth], paper_first_paper_year)
        else:
            css_first_paper_year[auth] = paper_first_paper_year
        if auth in css_citation_count:
            css_citation_count[auth] += [css_df.iloc[i]['citationCount']]
        else:
            css_citation_count[auth] = [css_df.iloc[i]['citationCount']]
#make dictionary with keys as authorId and values as field using css_df pandas dataframe:
for i in range(len(css_df)):
    css_author_field[author_data.iloc[i]['authorId']] = author_data.iloc[i]['field']
css_author_field = {node: css_author_field[node] for node in css_author_field if node in G.nodes()}

#add attributes to nodes:
for node in G.nodes():
    node = str(node)
    author_name = [author_data[author_data['authorId'] == node]['name'].values[0]] if not author_data[author_data['authorId'] == node]['name'].empty else []
    author_aliases = author_data[author_data['authorId'] == node]['aliases'].values.tolist()[0] if not author_data[author_data['authorId'] == node]['aliases'].empty else []
    author_aliases = ast.literal_eval(author_aliases) if (not str(author_aliases) == 'nan' and not author_aliases == []) else str(author_aliases)
    author_aliases = [author_aliases] if isinstance(author_aliases,str) else author_aliases
    author_fullname = author_name + author_aliases
    if author_fullname != []:
        author_fullname = max(author_fullname + author_aliases)
        css_author_fullname[node] = author_fullname
    css_citation_count_median[node] = np.median(np.sort(css_citation_count[node]))
    css_num_papers[node] = len(css_citation_count[node])


In [343]:
nx.set_node_attributes(G, name = 'name', values = css_author_fullname)
nx.set_node_attributes(G, name = 'citation_count', values = css_citation_count_median)
nx.set_node_attributes(G, name = 'css_papers', values = css_num_papers)
nx.set_node_attributes(G, name = 'top field', values = css_author_field)
nx.set_node_attributes(G, name = 'first_paper_year', values = css_first_paper_year)


In [344]:
G.nodes()

NodeView(('71958542', '50657181', '1573630453', '1573630578', '48988567', '2053577568', '145883179', '2703478', '1405628931', '1684687', '32261909', '1821014', '145752168', '4902230', '46590719', '3482524', '31485385', '148176055', '2165530', '48569628', '89090110', '143786158', '1752093', '2325084', '1744808', '3076440', '1701341', '3145906', '51435753', '151467651', '49305855', '48245618', '6673771', '2557324', '50663909', '33769943', '30912359', '144386157', '144982806', '1780833', '1685102', '1693341', '1739490', '34996452', '3371403', '145918049', '144441581', '2551214', '46950454', '3271584', '2383424', '101495832', '2107700135', '143900982', '2007521700', '2118744436', '1436038016', '1720151', '48418937', '1746034', '144252495', '1755938', '1779048', '145621251', '1690690', '1851304', '2145946569', '1737486', '3244651', '1826736', '2067699', '2349638', '2038139', '46726748', '1730404', '2836702', '1818812', '80273596', '2202543826', '2685443', '49446819', '41156871', '144728530'

In [348]:
G.edges('71958542')

EdgeDataView([('71958542', '50657181'), ('71958542', '1573630453'), ('71958542', '1573630578'), ('71958542', '48988567'), ('71958542', '2053577568')])

In [349]:
import pickle
pickle.dump(G, open(r'C:\Users\khali\OneDrive - Danmarks Tekniske Universitet\6. semester\Computational Social Science\Computational-Social-Science-Exercises\week4graph.pickle', 'wb'))


In [228]:
G = pickle.load(open('week4graph.pickle', 'rb'))

,Unnamed: 0.1,Unnamed: 0,paperId,title,year,externalId.DOI,citationCount,authorIds,fields,author_field
0,0,263,967628083778f90dc76f61b4b1a010a24dbae744,DESIGN OF A CROSS-BOUNDARY WARNING SYSTEM FOR ...,2019.0,"{'MAG': '2997661792', 'DOI': '10.35633/inmateh...",2,"['71958542', '50657181', '1573630453', '157363...",NaN,"['Economics', 'Engineering', 'Computer Science..."
1,1,1672,c45bb5875ff5922923812fe70c7138788e4e4c07,Monitoring of the Venezuelan exodus through Fa...,2020.0,"{'MAG': '3007572408', 'PubMedCentral': '703495...",30,"['145883179', '2703478', '1405628931', '1684687']",['Political Science'],"['Economics', 'Computer Science', 'Environment..."
2,2,1673,c85026963493ae17bce067b1a940d03dd1b9d50a,Correction: Monitoring of the Venezuelan exodu...,2020.0,"{'PubMedCentral': '7064226', 'MAG': '301145846...",4,"['145883179', '2703478', '1405628931', '1684687']",['Economics'],"['Economics', 'Computer Science', 'Environment..."
3,3,1676,6b5189161d69ae6a1d1063af76a6437eb82fbbcf,Migrant mobility flows characterized with digi...,2019.0,"{'MAG': '3012900549', 'DBLP': 'journals/corr/a...",19,"['145883179', '32261909', '1821014']",['Economics'],"['Economics', 'Economics', 'Computer Science']"
4,4,3607,28fcc6f52e4649487e51ecfc65bd83dc9715a325,Informed consent for linking survey and social...,2021.0,"{'MAG': '3146101303', 'DOI': '10.29173/IQ988',...",4,"['145752168', '4902230', '46590719', '3482524'...",['Computer Science'],"['Sociology', 'Computer Science', 'Psychology'..."
...,...,...,...,...,...,...,...,...,...,...
3218,3218,2225388,0b815b14a14a2de64386e4326750b4896d998555,Building Research Infrastructures to Study Dig...,2021.0,"{'MAG': '3197955738', 'DOI': '10.1017/S1049096...",5,"['2488381', '104244453']",['Computer Science'],"['Computer Science', 'Political Science']"
3219,3219,2228513,17cbcc0542a99b8b649df6968b28f93eb331c135,XiaoMu: an AI-driven assistant for MOOCs,2021.0,"{'DBLP': 'journals/chinaf/Song0LZWFZ21', 'MAG'...",1,"['46199760', '3355580']",['Education'],"['Computer Science', 'Economics']"
3220,3220,2228541,2ac00fc11d4ee9e709a5bcde57d35f75b5df2edd,Understanding Dropouts in MOOCs,2019.0,"{'DBLP': 'conf/aaai/Feng0L19', 'MAG': '2905464...",108,"['46199760', '3355580']",['Computer Science'],"['Computer Science', 'Economics']"
3221,3221,2228550,a8b43f3389dfbabd078bac406aa79bb88af53cb2,Infer Implicit Contexts in Real-time Online-to...,2019.0,"{'MAG': '3104763847', 'DBLP': 'conf/kdd/Ding0L...",7,"['46199760', '3355580']",['Computer Science'],"['Computer Science', 'Economics']"


In [206]:
author_data[author_data['authorId'] == node]['name']

Series([], Name: name, dtype: object)

In [205]:
author_data

'104244453'

In [209]:
# loop through author data and find row where authorId == node:
for i in range(len(author_data)):
    if author_data.iloc[i]['authorId'] == node:
        print(author_data.iloc[i])